Installation et import de PySpark

In [1]:
!pip install -q pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 54.4 MB/s 


In [2]:
import pyspark

Connexion Google Drive

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
!ls "/content/drive/MyDrive/Gaetan_Travail/ML/MP_Data/Test Data Science[1]"

df_a_utiliser.csv  model_logReg.pk  nba_logreg.csv
model_logReg2.pk   model.pk	    test.py


In [9]:
!ls "/content/"

drive  nba_logreg.csv  sample_data


In [5]:
path_file = "/content/df_a_utiliser.csv"

Tester lecture csv avec PySpark

In [23]:
# Pandas:
import pandas as pd
path_file = "/content/drive/MyDrive/Gaetan_Travail/ML/MP_Data/Test Data Science[1]/df_a_utiliser.csv"
pd.read_csv(path_file).head()

,GP,PTS,FG%,3P%,FT%,OREB,DREB,AST,STL,BLK,TOV,TARGET_5Yrs
0,36,2.128232,34.7,25.0,69.9,0.530628,1.481605,1.064711,0.336472,0.336472,0.832909,0.0
1,35,2.104134,29.6,23.5,76.5,0.405465,1.098612,1.547563,0.741937,0.405465,0.955511,0.0
2,74,1.824549,42.2,24.4,67.0,0.405465,0.993252,0.693147,0.405465,0.262364,0.693147,0.0
3,58,1.902108,42.6,22.6,68.9,0.693147,0.641854,0.587787,0.470004,0.095310,0.693147,1.0
4,48,1.704748,52.4,0.0,67.4,0.693147,0.916291,0.262364,0.262364,0.336472,0.587787,1.0


In [ ]:
'''
Pour utiliser Spark, il faut toujours lancer une session Spark !!
'''

In [205]:
# Creation d'une Spark session
from pyspark.sql import SparkSession

In [206]:
# Instancier une session, en lui donnant le nom souhaite
spark = SparkSession.builder.appName("PySpark_test").getOrCreate()

In [ ]:
'''
En local -> qu'une seule instance (juste Master)
Cloud -> possible de créer plusieurs instances (Master, cluster1, cluster2, etc.)
'''

In [207]:
spark

In [15]:
# Lecture d'un dataset
# fichier nba_logreg.csv a importer
df_pyspark = spark.read.csv("nba_logreg.csv")

In [17]:
df_pyspark.show()

+----------------+---+----+----+---+----+----+-------+---+----+----+----+----+----+----+----+----+----+----+----+-----------+
|             _c0|_c1| _c2| _c3|_c4| _c5| _c6|    _c7|_c8| _c9|_c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|       _c20|
+----------------+---+----+----+---+----+----+-------+---+----+----+----+----+----+----+----+----+----+----+----+-----------+
|            Name| GP| MIN| PTS|FGM| FGA| FG%|3P Made|3PA| 3P%| FTM| FTA| FT%|OREB|DREB| REB| AST| STL| BLK| TOV|TARGET_5Yrs|
|  Brandon Ingram| 36|27.4| 7.4|2.6| 7.6|34.7|    0.5|2.1|25.0| 1.6| 2.3|69.9| 0.7| 3.4| 4.1| 1.9| 0.4| 0.4| 1.3|        0.0|
| Andrew Harrison| 35|26.9| 7.2|2.0| 6.7|29.6|    0.7|2.8|23.5| 2.6| 3.4|76.5| 0.5| 2.0| 2.4| 3.7| 1.1| 0.5| 1.6|        0.0|
|  JaKarr Sampson| 74|15.3| 5.2|2.0| 4.7|42.2|    0.4|1.7|24.4| 0.9| 1.3|67.0| 0.5| 1.7| 2.2| 1.0| 0.5| 0.3| 1.0|        0.0|
|     Malik Sealy| 58|11.6| 5.7|2.3| 5.5|42.6|    0.1|0.5|22.6| 0.9| 1.3|68.9| 1.0| 0.9| 1.9| 0.8| 0.6| 0.1| 1.0|     

In [21]:
# afficher noms colonnes a la place de _c0, _c1, etc.

# 1ere ligne considérée comme noms colonnes (headers)
df_pyspark = spark.read.option("header", "true").csv("nba_logreg.csv")

In [28]:
# type de df pandas
print(f"Type pandas df:  {type(pd.read_csv(path_file))}\n")

# type de df_pyspark

print(f"Type PySpark df: {type(df_pyspark)}")

Type pandas df:  <class 'pandas.core.frame.DataFrame'>

Type PySpark df: <class 'pyspark.sql.dataframe.DataFrame'>


In [30]:
# 3 premieres lignes
df_pyspark.head(3)

[Row(Name='Brandon Ingram', GP='36', MIN='27.4', PTS='7.4', FGM='2.6', FGA='7.6', FG%='34.7', 3P Made='0.5', 3PA='2.1', 3P%='25.0', FTM='1.6', FTA='2.3', FT%='69.9', OREB='0.7', DREB='3.4', REB='4.1', AST='1.9', STL='0.4', BLK='0.4', TOV='1.3', TARGET_5Yrs='0.0'),
 Row(Name='Andrew Harrison', GP='35', MIN='26.9', PTS='7.2', FGM='2.0', FGA='6.7', FG%='29.6', 3P Made='0.7', 3PA='2.8', 3P%='23.5', FTM='2.6', FTA='3.4', FT%='76.5', OREB='0.5', DREB='2.0', REB='2.4', AST='3.7', STL='1.1', BLK='0.5', TOV='1.6', TARGET_5Yrs='0.0'),
 Row(Name='JaKarr Sampson', GP='74', MIN='15.3', PTS='5.2', FGM='2.0', FGA='4.7', FG%='42.2', 3P Made='0.4', 3PA='1.7', 3P%='24.4', FTM='0.9', FTA='1.3', FT%='67.0', OREB='0.5', DREB='1.7', REB='2.2', AST='1.0', STL='0.5', BLK='0.3', TOV='1.0', TARGET_5Yrs='0.0')]

In [31]:
# informations par rapport aux colonnes
# equivalent de df.info() sous pandas
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P Made: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FTM: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- OREB: string (nullable = true)
 |-- DREB: string (nullable = true)
 |-- REB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- TARGET_5Yrs: string (nullable = true)



# 2nd partie:
- PySpark DataFrame & Reading the Dataset
- Checking the Datatypes of the Columns (Schema)
- Selecting Columns and Indexing
- Check Describe option similar to Pandas
- Adding & Dropping Columns

# PySpark DataFrame & Reading the Dataset

In [32]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("PySpark_test").getOrCreate()
spark

In [35]:
# read the dataset
# par defaut, toutes les colonnes sont des string
df_pyspark = spark.read.option("header", "true").csv("nba_logreg.csv")

In [36]:
# Check the schema
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- GP: string (nullable = true)
 |-- MIN: string (nullable = true)
 |-- PTS: string (nullable = true)
 |-- FGM: string (nullable = true)
 |-- FGA: string (nullable = true)
 |-- FG%: string (nullable = true)
 |-- 3P Made: string (nullable = true)
 |-- 3PA: string (nullable = true)
 |-- 3P%: string (nullable = true)
 |-- FTM: string (nullable = true)
 |-- FTA: string (nullable = true)
 |-- FT%: string (nullable = true)
 |-- OREB: string (nullable = true)
 |-- DREB: string (nullable = true)
 |-- REB: string (nullable = true)
 |-- AST: string (nullable = true)
 |-- STL: string (nullable = true)
 |-- BLK: string (nullable = true)
 |-- TOV: string (nullable = true)
 |-- TARGET_5Yrs: string (nullable = true)



In [37]:
# changer le type des colonnes
# ajouter "inferSchema=True" dans .csv()
df_pyspark = spark.read.option("header", "true").csv(
    "nba_logreg.csv",
    inferSchema=True
)

In [38]:
# Check the schema
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- MIN: double (nullable = true)
 |-- PTS: double (nullable = true)
 |-- FGM: double (nullable = true)
 |-- FGA: double (nullable = true)
 |-- FG%: double (nullable = true)
 |-- 3P Made: double (nullable = true)
 |-- 3PA: double (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FTM: double (nullable = true)
 |-- FTA: double (nullable = true)
 |-- FT%: double (nullable = true)
 |-- OREB: double (nullable = true)
 |-- DREB: double (nullable = true)
 |-- REB: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- STL: double (nullable = true)
 |-- BLK: double (nullable = true)
 |-- TOV: double (nullable = true)
 |-- TARGET_5Yrs: double (nullable = true)



In [42]:
# autre facon de lire les données
# line de code plus compacte, en se passant de option()
df_pyspark = spark.read.csv(
    "nba_logreg.csv",
    header=True,
    inferSchema=True
)

# Checking the Datatypes of the Columns (Schema)

In [57]:
df_pyspark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- MIN: double (nullable = true)
 |-- PTS: double (nullable = true)
 |-- FGM: double (nullable = true)
 |-- FGA: double (nullable = true)
 |-- FG%: double (nullable = true)
 |-- 3P Made: double (nullable = true)
 |-- 3PA: double (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FTM: double (nullable = true)
 |-- FTA: double (nullable = true)
 |-- FT%: double (nullable = true)
 |-- OREB: double (nullable = true)
 |-- DREB: double (nullable = true)
 |-- REB: double (nullable = true)
 |-- AST: double (nullable = true)
 |-- STL: double (nullable = true)
 |-- BLK: double (nullable = true)
 |-- TOV: double (nullable = true)
 |-- TARGET_5Yrs: double (nullable = true)



NoneType

In [41]:
# type = dataFrame (data structure, dans laquelle on peut operer
# differentes operations sur les donnees)
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

# Selecting Columns and Indexing

In [54]:
# comment obtenir toutes les colonnes ?
df_pyspark.columns

['Name',
 'GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3P Made',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'TARGET_5Yrs']

In [45]:
# 3 premieres lignes
# liste format (pandas -> df)
df_pyspark.head(3)

[Row(Name='Brandon Ingram', GP=36, MIN=27.4, PTS=7.4, FGM=2.6, FGA=7.6, FG%=34.7, 3P Made=0.5, 3PA=2.1, 3P%=25.0, FTM=1.6, FTA=2.3, FT%=69.9, OREB=0.7, DREB=3.4, REB=4.1, AST=1.9, STL=0.4, BLK=0.4, TOV=1.3, TARGET_5Yrs=0.0),
 Row(Name='Andrew Harrison', GP=35, MIN=26.9, PTS=7.2, FGM=2.0, FGA=6.7, FG%=29.6, 3P Made=0.7, 3PA=2.8, 3P%=23.5, FTM=2.6, FTA=3.4, FT%=76.5, OREB=0.5, DREB=2.0, REB=2.4, AST=3.7, STL=1.1, BLK=0.5, TOV=1.6, TARGET_5Yrs=0.0),
 Row(Name='JaKarr Sampson', GP=74, MIN=15.3, PTS=5.2, FGM=2.0, FGA=4.7, FG%=42.2, 3P Made=0.4, 3PA=1.7, 3P%=24.4, FTM=0.9, FTA=1.3, FT%=67.0, OREB=0.5, DREB=1.7, REB=2.2, AST=1.0, STL=0.5, BLK=0.3, TOV=1.0, TARGET_5Yrs=0.0)]

In [49]:
# comment selectionner une colonne en particulier ?
# ex : ne selectionner que la colonne "GP"
df_pyspark.select("GP").show(3)

+---+
| GP|
+---+
| 36|
| 35|
| 74|
+---+
only showing top 3 rows



In [50]:
# show() fonctionne car l'objet retourne par select() est un df
type(df_pyspark.select("GP"))

pyspark.sql.dataframe.DataFrame

In [86]:
# plusieurs colonnes
# ex : GP et PTS
df_pyspark.select("GP", "PTS").show(3)

+---+---+
| GP|PTS|
+---+---+
| 36|7.4|
| 35|7.2|
| 74|5.2|
+---+---+
only showing top 3 rows



In [66]:
# afficher 1 ligne (ici la troisieme)
df_pyspark.collect()[2]

Row(Name='JaKarr Sampson', GP=74, MIN=15.3, PTS=5.2, FGM=2.0, FGA=4.7, FG%=42.2, 3P Made=0.4, 3PA=1.7, 3P%=24.4, FTM=0.9, FTA=1.3, FT%=67.0, OREB=0.5, DREB=1.7, REB=2.2, AST=1.0, STL=0.5, BLK=0.3, TOV=1.0, TARGET_5Yrs=0.0)

In [67]:
# afficher les 3 premieres lignes
df_pyspark.collect()[:3]

[Row(Name='Brandon Ingram', GP=36, MIN=27.4, PTS=7.4, FGM=2.6, FGA=7.6, FG%=34.7, 3P Made=0.5, 3PA=2.1, 3P%=25.0, FTM=1.6, FTA=2.3, FT%=69.9, OREB=0.7, DREB=3.4, REB=4.1, AST=1.9, STL=0.4, BLK=0.4, TOV=1.3, TARGET_5Yrs=0.0),
 Row(Name='Andrew Harrison', GP=35, MIN=26.9, PTS=7.2, FGM=2.0, FGA=6.7, FG%=29.6, 3P Made=0.7, 3PA=2.8, 3P%=23.5, FTM=2.6, FTA=3.4, FT%=76.5, OREB=0.5, DREB=2.0, REB=2.4, AST=3.7, STL=1.1, BLK=0.5, TOV=1.6, TARGET_5Yrs=0.0),
 Row(Name='JaKarr Sampson', GP=74, MIN=15.3, PTS=5.2, FGM=2.0, FGA=4.7, FG%=42.2, 3P Made=0.4, 3PA=1.7, 3P%=24.4, FTM=0.9, FTA=1.3, FT%=67.0, OREB=0.5, DREB=1.7, REB=2.2, AST=1.0, STL=0.5, BLK=0.3, TOV=1.0, TARGET_5Yrs=0.0)]

In [68]:
# ou
df_pyspark.rdd.take(3)

[Row(Name='Brandon Ingram', GP=36, MIN=27.4, PTS=7.4, FGM=2.6, FGA=7.6, FG%=34.7, 3P Made=0.5, 3PA=2.1, 3P%=25.0, FTM=1.6, FTA=2.3, FT%=69.9, OREB=0.7, DREB=3.4, REB=4.1, AST=1.9, STL=0.4, BLK=0.4, TOV=1.3, TARGET_5Yrs=0.0),
 Row(Name='Andrew Harrison', GP=35, MIN=26.9, PTS=7.2, FGM=2.0, FGA=6.7, FG%=29.6, 3P Made=0.7, 3PA=2.8, 3P%=23.5, FTM=2.6, FTA=3.4, FT%=76.5, OREB=0.5, DREB=2.0, REB=2.4, AST=3.7, STL=1.1, BLK=0.5, TOV=1.6, TARGET_5Yrs=0.0),
 Row(Name='JaKarr Sampson', GP=74, MIN=15.3, PTS=5.2, FGM=2.0, FGA=4.7, FG%=42.2, 3P Made=0.4, 3PA=1.7, 3P%=24.4, FTM=0.9, FTA=1.3, FT%=67.0, OREB=0.5, DREB=1.7, REB=2.2, AST=1.0, STL=0.5, BLK=0.3, TOV=1.0, TARGET_5Yrs=0.0)]

# Check Describe option similar to Pandas

In [69]:
df_pyspark.dtypes

[('Name', 'string'),
 ('GP', 'int'),
 ('MIN', 'double'),
 ('PTS', 'double'),
 ('FGM', 'double'),
 ('FGA', 'double'),
 ('FG%', 'double'),
 ('3P Made', 'double'),
 ('3PA', 'double'),
 ('3P%', 'double'),
 ('FTM', 'double'),
 ('FTA', 'double'),
 ('FT%', 'double'),
 ('OREB', 'double'),
 ('DREB', 'double'),
 ('REB', 'double'),
 ('AST', 'double'),
 ('STL', 'double'),
 ('BLK', 'double'),
 ('TOV', 'double'),
 ('TARGET_5Yrs', 'double')]

In [71]:
df_pyspark.describe().show()

+-------+-------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-------------------+
|summary|         Name|                GP|              MIN|               PTS|               FGM|              FGA|              FG%|            3P Made|               3PA|               3P%|               FTM|               FTA|               FT%|              OREB|              DREB|               REB|               AST|                STL|               BLK|               TOV|        TARGET_5Yrs|
+-------+-------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+------------------+

# Adding & Dropping Columns

In [77]:
# Adding columns in dataFrame

# col name, value (col <- df_pyspark[...] et non df)
df_pyspark = df_pyspark.withColumn("test col", df_pyspark["GP"]*2)

# afficher 3 premieres lignes et que col name, gp et test col
df_pyspark.select(["Name", "GP", "test col"]).show(3)

+---------------+---+--------+
|           Name| GP|test col|
+---------------+---+--------+
| Brandon Ingram| 36|      72|
|Andrew Harrison| 35|      70|
| JaKarr Sampson| 74|     148|
+---------------+---+--------+
only showing top 3 rows



In [83]:
# Dropping 1 col in dataFrame
df_pyspark = df_pyspark.drop("test col")

df_pyspark.show(3)

+---------------+---+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
|           Name| GP| MIN|PTS|FGM|FGA| FG%|3P Made|3PA| 3P%|FTM|FTA| FT%|OREB|DREB|REB|AST|STL|BLK|TOV|TARGET_5Yrs|
+---------------+---+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
| Brandon Ingram| 36|27.4|7.4|2.6|7.6|34.7|    0.5|2.1|25.0|1.6|2.3|69.9| 0.7| 3.4|4.1|1.9|0.4|0.4|1.3|        0.0|
|Andrew Harrison| 35|26.9|7.2|2.0|6.7|29.6|    0.7|2.8|23.5|2.6|3.4|76.5| 0.5| 2.0|2.4|3.7|1.1|0.5|1.6|        0.0|
| JaKarr Sampson| 74|15.3|5.2|2.0|4.7|42.2|    0.4|1.7|24.4|0.9|1.3|67.0| 0.5| 1.7|2.2|1.0|0.5|0.3|1.0|        0.0|
+---------------+---+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
only showing top 3 rows



In [84]:
# Dropping 3 col
df_pyspark.drop("GP", "MIN", "FGA").show(3)

+---------------+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
|           Name|PTS|FGM| FG%|3P Made|3PA| 3P%|FTM|FTA| FT%|OREB|DREB|REB|AST|STL|BLK|TOV|TARGET_5Yrs|
+---------------+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
| Brandon Ingram|7.4|2.6|34.7|    0.5|2.1|25.0|1.6|2.3|69.9| 0.7| 3.4|4.1|1.9|0.4|0.4|1.3|        0.0|
|Andrew Harrison|7.2|2.0|29.6|    0.7|2.8|23.5|2.6|3.4|76.5| 0.5| 2.0|2.4|3.7|1.1|0.5|1.6|        0.0|
| JaKarr Sampson|5.2|2.0|42.2|    0.4|1.7|24.4|0.9|1.3|67.0| 0.5| 1.7|2.2|1.0|0.5|0.3|1.0|        0.0|
+---------------+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
only showing top 3 rows



In [87]:
# Renaming a col (pas de inplace, donc df = df.with... pour save modifications)
df_pyspark.withColumnRenamed("GP", "New_GP").show(3)

+---------------+------+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
|           Name|New_GP| MIN|PTS|FGM|FGA| FG%|3P Made|3PA| 3P%|FTM|FTA| FT%|OREB|DREB|REB|AST|STL|BLK|TOV|TARGET_5Yrs|
+---------------+------+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
| Brandon Ingram|    36|27.4|7.4|2.6|7.6|34.7|    0.5|2.1|25.0|1.6|2.3|69.9| 0.7| 3.4|4.1|1.9|0.4|0.4|1.3|        0.0|
|Andrew Harrison|    35|26.9|7.2|2.0|6.7|29.6|    0.7|2.8|23.5|2.6|3.4|76.5| 0.5| 2.0|2.4|3.7|1.1|0.5|1.6|        0.0|
| JaKarr Sampson|    74|15.3|5.2|2.0|4.7|42.2|    0.4|1.7|24.4|0.9|1.3|67.0| 0.5| 1.7|2.2|1.0|0.5|0.3|1.0|        0.0|
+---------------+------+----+---+---+---+----+-------+---+----+---+---+----+----+----+---+---+---+---+---+-----------+
only showing top 3 rows



# Partie 3: PySpark Handling Missing Values
- Dropping columns or rows
- Various parameter in dropping functionalities
- Handling missing values by mean, median and mode

In [204]:
# shape function
def spark_shape(self):
  return (self.count(), len(self.columns))

In [89]:
# si debut projet -> start a pyspark session!
'''
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark_test").getOrCreate()
'''

spark

In [92]:
df_pyspark = spark.read.csv(
    "test_pyspark_1.csv",
    header=True,
    inferSchema=True
)

df_pyspark.show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  Krish|  31|        10| 30000|
|    Sud|  30|         8| 25000|
|  Sunny|  29|         4| 20000|
|   Paul|  24|         3| 20000|
| Harsha|  21|         1| 15000|
|Shubham|  23|         2| 18000|
| Mahesh|null|      null| 40000|
|   null|  34|        10| 38000|
|   null|  36|      null|  null|
+-------+----+----------+------+



In [95]:
df_pyspark

DataFrame[Name: string, Age: int, Experience: int, Salary: int]

In [94]:
df_pyspark.describe().show()

+-------+------+------------------+------------------+-----------------+
|summary|  Name|               Age|        Experience|           Salary|
+-------+------+------------------+------------------+-----------------+
|  count|     7|                 8|                 7|                8|
|   mean|  null|              28.5| 5.428571428571429|          25750.0|
| stddev|  null|5.3718844791323335|3.8234863173611093|9361.776388210581|
|    min|Harsha|                21|                 1|            15000|
|    max| Sunny|                36|                10|            40000|
+-------+------+------------------+------------------+-----------------+



# Dropping columns or rows

In [96]:
# Dropping col
df_pyspark.drop("Name").show()

+----+----------+------+
| Age|Experience|Salary|
+----+----------+------+
|  31|        10| 30000|
|  30|         8| 25000|
|  29|         4| 20000|
|  24|         3| 20000|
|  21|         1| 15000|
|  23|         2| 18000|
|null|      null| 40000|
|  34|        10| 38000|
|  36|      null|  null|
+----+----------+------+



In [ ]:
# Dropping NaN values

In [100]:
# df original
df_pyspark.show()

print(f"df shape = ({df_pyspark.count()}, {len(df_pyspark.columns)})")

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  Krish|  31|        10| 30000|
|    Sud|  30|         8| 25000|
|  Sunny|  29|         4| 20000|
|   Paul|  24|         3| 20000|
| Harsha|  21|         1| 15000|
|Shubham|  23|         2| 18000|
| Mahesh|null|      null| 40000|
|   null|  34|        10| 38000|
|   null|  36|      null|  null|
+-------+----+----------+------+

df shape = (9, 4)


In [105]:
# df_pyspark.na -> drop, fill or replace
df_pyspark.na.drop().show()

print(f"df shape = {spark_shape(df_pyspark.na.drop())}")

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+

df shape = (6, 4)


# Various parameter in dropping functionalities

In [106]:
# drop function
# how

# how == any (dafault) -> si au moins 1 null/nan dans ligne -> suppr
# how == all -> suppr si lignes ne contient que des null
# si une valeur de la ligne =/= null => pas suppr
df_pyspark.na.drop(how="all").show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  Krish|  31|        10| 30000|
|    Sud|  30|         8| 25000|
|  Sunny|  29|         4| 20000|
|   Paul|  24|         3| 20000|
| Harsha|  21|         1| 15000|
|Shubham|  23|         2| 18000|
| Mahesh|null|      null| 40000|
|   null|  34|        10| 38000|
|   null|  36|      null|  null|
+-------+----+----------+------+



In [118]:
# drop function
# threshold

# ligne contient x valeurs non nulles
# si x >= thresh, alors ligne consevee
# si x <  thresh, alors ligne supprimee
df_pyspark.na.drop(thresh=2).show()

+-------+----+----------+------+
|   Name| Age|Experience|Salary|
+-------+----+----------+------+
|  Krish|  31|        10| 30000|
|    Sud|  30|         8| 25000|
|  Sunny|  29|         4| 20000|
|   Paul|  24|         3| 20000|
| Harsha|  21|         1| 15000|
|Shubham|  23|         2| 18000|
| Mahesh|null|      null| 40000|
|   null|  34|        10| 38000|
+-------+----+----------+------+



In [122]:
# drop function
# subset

# enlever nan values seulement des cols age et salary
df_pyspark.na.drop(subset=["Age", "Salary"]).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
|   null| 34|        10| 38000|
+-------+---+----------+------+



In [130]:
# Filling the missing values

# arguments -> value et subset
# si col string -> indiquer un string (ex: "Missing Value")
# si col int ou float -> iniquer un nombre (ex: 0)
df_pyspark.na.fill(0, ["Age", "Experience"]).show() # 2 col
df_pyspark.na.fill(0).show() # toutes les col int ou float

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
| Mahesh|  0|         0| 40000|
|   null| 34|        10| 38000|
|   null| 36|         0|  null|
+-------+---+----------+------+

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
| Mahesh|  0|         0| 40000|
|   null| 34|        10| 38000|
|   null| 36|         0|     0|
+-------+---+----------+------+



# Handling missing values by mean, median and mode

In [135]:
# Imputer function
from pyspark.ml.feature import Imputer

list_cols = ["Age", "Experience", "Salary"]

# ici mean, mais aussi possible pour median, mode, etc.
imputer = Imputer(
    inputCols = list_cols,
    outputCols = [f"{x}_imputed" for x in list_cols]
).setStrategy("mean")

In [136]:
# remplacer missing values des col int/float par la moyenne de ces cols

# Adding imputation cols to df
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-------+----+----------+------+-----------+------------------+--------------+
|   Name| Age|Experience|Salary|Age_imputed|Experience_imputed|Salary_imputed|
+-------+----+----------+------+-----------+------------------+--------------+
|  Krish|  31|        10| 30000|         31|                10|         30000|
|    Sud|  30|         8| 25000|         30|                 8|         25000|
|  Sunny|  29|         4| 20000|         29|                 4|         20000|
|   Paul|  24|         3| 20000|         24|                 3|         20000|
| Harsha|  21|         1| 15000|         21|                 1|         15000|
|Shubham|  23|         2| 18000|         23|                 2|         18000|
| Mahesh|null|      null| 40000|         28|                 5|         40000|
|   null|  34|        10| 38000|         34|                10|         38000|
|   null|  36|      null|  null|         36|                 5|         25750|
+-------+----+----------+------+-----------+--------

# Partie 4: PySpark DataFrames - Filter operation
- Filter Operations
- &, |, ==
- ~

In [144]:
df_pyspark = df_pyspark.na.drop()
df_pyspark.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



# Filter Operations

In [147]:
# Ex: People salaries less than or equal to 20 000
df_pyspark.filter("Salary <= 20000").show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [148]:
df_pyspark.filter("Salary <= 20000").select(["Name", "Age"]).show()

+-------+---+
|   Name|Age|
+-------+---+
|  Sunny| 29|
|   Paul| 24|
| Harsha| 21|
|Shubham| 23|
+-------+---+



In [149]:
# autre facon
df_pyspark.filter(df_pyspark["Salary"] <= 20000).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



# &, |, ==

In [152]:
# 2 conditions
df_pyspark.filter((df_pyspark["Salary"] <= 20000) \
                  & (df_pyspark["Salary"] >= 18000)).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [154]:
# 2 conditions
df_pyspark.filter((df_pyspark["Salary"] <= 20000) \
                  | (df_pyspark["Salary"] >= 18000)).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



 # ~

In [155]:
df_pyspark.filter(df_pyspark["Salary"] <= 20000).show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [157]:
# ~ -> not condition
# i.e. tout ce qui NE satisfait PAS les conditions
df_pyspark.filter(~(df_pyspark["Salary"] <= 20000)).show()

+-----+---+----------+------+
| Name|Age|Experience|Salary|
+-----+---+----------+------+
|Krish| 31|        10| 30000|
|  Sud| 30|         8| 25000|
+-----+---+----------+------+



# Partie 5: PySpark GroupBy and Aggregate Functions

In [208]:
df_pyspark = spark.read.csv(
    "test_pyspark_5.csv",
    header=True,
    inferSchema=True
)

df_pyspark.show()

+---------+------------+------+
|     Name|Departements|Salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [209]:
print(f"PySpark df shape = {spark_shape(df_pyspark)}\n")

df_pyspark.printSchema()

df_pyspark.describe().show()

PySpark df shape = (10, 3)

root
 |-- Name: string (nullable = true)
 |-- Departements: string (nullable = true)
 |-- Salary: integer (nullable = true)

+-------+-----+------------+-----------------+
|summary| Name|Departements|           Salary|
+-------+-----+------------+-----------------+
|  count|   10|          10|               10|
|   mean| null|        null|           7300.0|
| stddev| null|        null|5396.500923952689|
|    min|Krish|    Big Data|             2000|
|    max|Sunny|         IOT|            20000|
+-------+-----+------------+-----------------+



In [210]:
# groupby name, mean salaries
df_pyspark.groupBy("Name").mean("Salary").show()

+---------+------------------+
|     Name|       avg(Salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



In [211]:
# How has the maximum salary?
df_pyspark.groupBy("Name").sum("Salary").orderBy(
    "sum(Salary)",
    ascending=False
).show()

# aggregate function
# df_pyspark.groupBy("Name").agg({"Salary": "sum"}).show()

+---------+-----------+
|     Name|sum(Salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Krish|      19000|
|    Sunny|      12000|
|   Mahesh|       7000|
+---------+-----------+



In [212]:
# collect recupere les lignes au sein d'une liste

# attention si collect() -> va recuperer TOUTES les lignes
# peut occasionner des problemes de memoires...

# collect()[n] -> collect les n premieres lignes seulement

tmp = df_pyspark.groupBy("Name").sum("Salary").orderBy(
    "sum(Salary)",
    ascending=False
).collect()[0]

print(f"Max value:\nName = {tmp[0]}\nMax salary = {tmp[1]}")

Max value:
Name = Sudhanshu
Max salary = 35000


In [213]:
# autre facon
# rdd.take(n) -> n doit toujours etre specifie

tmp = df_pyspark.groupBy("Name").sum("Salary").orderBy(
    "sum(Salary)",
    ascending=False
).rdd.take(3)

tmp

[Row(Name='Sudhanshu', sum(Salary)=35000),
 Row(Name='Krish', sum(Salary)=19000),
 Row(Name='Sunny', sum(Salary)=12000)]

In [214]:
print(f"Max value:\nName = {tmp[0][0]}\nMax salary = {tmp[0][1]}")

Max value:
Name = Sudhanshu
Max salary = 35000


In [217]:
# number of emploies per department
# i.e. number of rows for each department
df_pyspark.groupBy("Departements").count().show()

+------------+-----+
|Departements|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



In [220]:
# aggregate functions

df_pyspark.agg({"Salary": "sum"}).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+



In [223]:
df_pyspark.agg({"Salary": "sum"}).collect()[0][0]

73000

In [ ]:
df_pyspark.groupBy("Name").sum("Salary").orderBy(
    "sum(Salary)",
    ascending=False
).show()

In [227]:
df_pyspark.groupBy("Name").agg({"Salary": "max"}).show()

+---------+-----------+
|     Name|max(Salary)|
+---------+-----------+
|Sudhanshu|      20000|
|    Sunny|      10000|
|    Krish|      10000|
|   Mahesh|       4000|
+---------+-----------+



# Partie 6: PySpark MLlib - Installation and Implementation

Exemples via le site [Spark MLlib](https://spark.apache.org/examples.html)
- RDD API: API bas niveau, originale
- DF API: API haut niveau, plus récente
- ML API: API haut niveau

In [233]:
df_train = spark.read.csv(
    "test_pyspark_1.csv",
    header=True,
    inferSchema=True
).na.drop()

df_train.show()

+-------+---+----------+------+
|   Name|Age|Experience|Salary|
+-------+---+----------+------+
|  Krish| 31|        10| 30000|
|    Sud| 30|         8| 25000|
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



In [237]:
print(f"df train shape = {spark_shape(df_train)}\n")

df_train.printSchema()

print(f"\nTraining columns: {df_train.columns}")

df train shape = (6, 4)

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)


Training columns: ['Name', 'Age', 'Experience', 'Salary']


# Data preprocessing

In [239]:
# creer un "VectorAssembler" qui permet d'imposer la meme structure
# aux donnees qu'elles viennent d'une machine distante, ou d'une autre

# Sous Spark, il faut grouper les feautres independantes pour creer un nouvelle
# objet (vector assembler) qui sera utilise par la suite
# ["Age", "Experience"] ----> new feature ----> independent features

from pyspark.ml.feature import VectorAssembler

featureAssembler = VectorAssembler(
    inputCols=["Age", "Experience"],  # features independantes
    outputCol="Independent Features" # nouvelle feature (pas une liste, sans [])
)

In [240]:
output = featureAssembler.transform(df_train)

In [241]:
output.show()

+-------+---+----------+------+--------------------+
|   Name|Age|Experience|Salary|Independent Features|
+-------+---+----------+------+--------------------+
|  Krish| 31|        10| 30000|         [31.0,10.0]|
|    Sud| 30|         8| 25000|          [30.0,8.0]|
|  Sunny| 29|         4| 20000|          [29.0,4.0]|
|   Paul| 24|         3| 20000|          [24.0,3.0]|
| Harsha| 21|         1| 15000|          [21.0,1.0]|
|Shubham| 23|         2| 18000|          [23.0,2.0]|
+-------+---+----------+------+--------------------+



In [242]:
print(f"Output columns: {output.columns}")

Output columns: ['Name', 'Age', 'Experience', 'Salary', 'Independent Features']


'Independent Features' == input feature

Salary == output feature (to be predected)

In [245]:
df = output.select(["Independent Features", "Salary"])

df.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



# Train test split

In [253]:
train_data, test_data = df.randomSplit([0.70, 0.30], seed=0)

print(f"train shape = {spark_shape(train_data)}")
print(f"test shape  = {spark_shape(test_data)}")

train shape = (4, 2)
test shape  = (2, 2)


# Regression model

In [255]:
from pyspark.ml.regression import LinearRegression

linReg_model = LinearRegression(
    featuresCol="Independent Features",
    labelCol="Salary",
    elasticNetParam=0,
    standardization=True
)

linReg_model = linReg_model.fit(train_data)

In [258]:
# Prediction
pred_results = linReg_model.evaluate(test_data)

In [273]:
# Print the coefficients and intercept for linear regression
print(f"Coefficients: {linReg_model.coefficients}")
print(f"Intercept: {linReg_model.intercept}")

# Metrics
print(f"\nRMSE: {pred_results.rootMeanSquaredError}")
print(f"MAE: {pred_results.meanAbsoluteError}")
print(f"r2: {pred_results.r2}")

Coefficients: [-298.24561403512723,1771.929824561437]
Intercept: 21561.403508772833

RMSE: 1934.9214988017682
MAE: 1929.824561403555
r2: 0.8502431517389888


In [274]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000|17070.175438596598|
|          [30.0,8.0]| 25000|26789.473684210512|
+--------------------+------+------------------+

